In [33]:
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en import English
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize.toktok import ToktokTokenizer
import re
import string

In [2]:
df = pd.read_csv("/Users/butler/Documents/GitHub/projects/tweets_climate_change/data/interim/third_eye_climate_words.csv")

In [3]:
df.head()

,Unnamed: 0,datetime,network,duration,network_info,text,date
0,1953,2017-09-07 17:10:00,BBCNEWS,5.0,BBCNEWS_20170907_170000_BBC_News_at_Six/start/600,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
1,2118,2017-09-07 18:37:00,BBCNEWS,4.0,BBCNEWS_20170907_180000_BBC_News/start/2220,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
2,2177,2017-09-07 19:13:00,BBCNEWS,4.0,BBCNEWS_20170907_190000_BBC_News/start/780,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
3,3153,2017-09-08 04:35:00,BBCNEWS,8.0,BBCNEWS_20170908_043000_World_Business_Report/...,HURRICANE COSTS. Irma expected to be among cos...,2017-09-08
4,9621,2017-09-11 10:52:00,BBCNEWS,21.0,BBCNEWS_20170911_100000_BBC_Newsroom_Live/star...,UK ECONOMIC OUTLOOK. ICAEW says investment cli...,2017-09-11


In [7]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [8]:
df = df.sort_values(by=["datetime"])

In [9]:
df.head()

,datetime,network,duration,network_info,text,date
0,2017-09-07 17:10:00,BBCNEWS,5.0,BBCNEWS_20170907_170000_BBC_News_at_Six/start/600,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
1,2017-09-07 18:37:00,BBCNEWS,4.0,BBCNEWS_20170907_180000_BBC_News/start/2220,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
2,2017-09-07 19:13:00,BBCNEWS,4.0,BBCNEWS_20170907_190000_BBC_News/start/780,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
3,2017-09-08 04:35:00,BBCNEWS,8.0,BBCNEWS_20170908_043000_World_Business_Report/...,HURRICANE COSTS. Irma expected to be among cos...,2017-09-08
4,2017-09-11 10:52:00,BBCNEWS,21.0,BBCNEWS_20170911_100000_BBC_Newsroom_Live/star...,UK ECONOMIC OUTLOOK. ICAEW says investment cli...,2017-09-11


In [10]:
df.head(20)

,datetime,network,duration,network_info,text,date
0,2017-09-07 17:10:00,BBCNEWS,5.0,BBCNEWS_20170907_170000_BBC_News_at_Six/start/600,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
1,2017-09-07 18:37:00,BBCNEWS,4.0,BBCNEWS_20170907_180000_BBC_News/start/2220,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
2,2017-09-07 19:13:00,BBCNEWS,4.0,BBCNEWS_20170907_190000_BBC_News/start/780,"Dr DANN MITCHELL. Climate Scientist, Bristol U...",2017-09-07
3,2017-09-08 04:35:00,BBCNEWS,8.0,BBCNEWS_20170908_043000_World_Business_Report/...,HURRICANE COSTS. Irma expected to be among cos...,2017-09-08
4,2017-09-11 10:52:00,BBCNEWS,21.0,BBCNEWS_20170911_100000_BBC_Newsroom_Live/star...,UK ECONOMIC OUTLOOK. ICAEW says investment cli...,2017-09-11
5,2017-09-11 10:53:00,BBCNEWS,15.0,BBCNEWS_20170911_100000_BBC_Newsroom_Live/star...,@ BBCBenThQMuson\nUK ECONOMIC OUTLOOK. ICAEW: ...,2017-09-11
6,2017-09-12 00:25:00,MSNBCW,38.0,MSNBCW_20170912_000000_All_In_With_Chris_Hayes...,FLORIDA REPUBLICAN: TIME TO TALK CLIMATE CHANG...,2017-09-12
7,2017-09-12 00:26:00,MSNBCW,62.0,MSNBCW_20170912_000000_All_In_With_Chris_Hayes...,FLORIDA REPUBLICAN: TIME TO TALK CLIMATE CHANGE,2017-09-12
8,2017-09-12 00:27:00,MSNBCW,35.0,MSNBCW_20170912_000000_All_In_With_Chris_Hayes...,FLORIDA REPUBLICAN: TIME TO TALK CLIMATE CHANG...,2017-09-12
9,2017-09-12 00:30:00,MSNBCW,11.0,MSNBCW_20170912_000000_All_In_With_Chris_Hayes...,FLORIDA REPUBLICAN: TIME TO TALK CLIMATE CHANGE,2017-09-12


In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
text = str(nlp(df.iloc[0,4]))

In [14]:
text

'Dr DANN MITCHELL. Climate Scientist, Bristol University'

In [15]:
sia = SentimentIntensityAnalyzer()

In [17]:
sia.polarity_scores(text)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [18]:
text = str(nlp(df.iloc[4,4]))

In [19]:
sia.polarity_scores(text)

{'neg': 0.0, 'neu': 0.836, 'pos': 0.164, 'compound': 0.6705}

In [21]:
text

'UK ECONOMIC OUTLOOK. ICAEW says investment climate good, costs low\\nUK ECONOMIC OUTLOOK. ICAEW: Businesses need to take leap of faith\\nUK ECONOMIC OUTLOOK. Group forecasting growth to slow to 1.6% in...'

In [24]:
text = str(nlp(df.iloc[13,4]))
sia.polarity_scores(text)

{'neg': 0.172, 'neu': 0.828, 'pos': 0.0, 'compound': -0.516}

In [25]:
text

'IRMA TRIGGERS SEVERE FLOODING IN FLORIDA\\u2019S LARGEST CITY\\nPENNY TAYLOR I CHAIR, COLLIERCOUNTYCOMMISSION\\nFLORIDA REPUBLICAN: TIME TO TALK CLIMATE CHANGE'

In [27]:
text = nlp(df.iloc[13,4])
text

IRMA TRIGGERS SEVERE FLOODING IN FLORIDA\u2019S LARGEST CITY\nPENNY TAYLOR I CHAIR, COLLIERCOUNTYCOMMISSION\nFLORIDA REPUBLICAN: TIME TO TALK CLIMATE CHANGE

In [28]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [29]:
text = nlp(df.iloc[13,4])
text

IRMA TRIGGERS SEVERE FLOODING IN FLORIDA\u2019S LARGEST CITY\nPENNY TAYLOR I CHAIR, COLLIERCOUNTYCOMMISSION\nFLORIDA REPUBLICAN: TIME TO TALK CLIMATE CHANGE

In [31]:
x = df["text"]

In [56]:
txt = []
for i in range(200,210):
    print(x[i])
    text = x[i].replace(string.punctuation, " ")
    print(text)
    text = re.sub(r'(!\,./:;\\)1+', ' ', text)
    print(text)
    text = text.replace("\\n", ' ')
    text = text.replace("\n", ' ')
    print(text)
    text = ''.join([i for i in text if not i.isdigit()])
    print(text)
    text = text.lower()
    print(text)
    text = re.sub(r'(\n)1+', ' ', text)
    
    txt.append(text)


TRUMP: CHINA GOT BETI'ER DEAL IN PARIS CLIMATE AGREEMENT. . Gov Jerry Brown (D) California\nCA GOV COMPARES TRUMP SUPPORTERS TO CAVE DWELLERS\nTRUMP: CHINA GOT BETTER DEAL IN PARIS CLIMATE AGREEMENT
TRUMP: CHINA GOT BETI'ER DEAL IN PARIS CLIMATE AGREEMENT. . Gov Jerry Brown (D) California\nCA GOV COMPARES TRUMP SUPPORTERS TO CAVE DWELLERS\nTRUMP: CHINA GOT BETTER DEAL IN PARIS CLIMATE AGREEMENT
TRUMP: CHINA GOT BETI'ER DEAL IN PARIS CLIMATE AGREEMENT. . Gov Jerry Brown (D) California\nCA GOV COMPARES TRUMP SUPPORTERS TO CAVE DWELLERS\nTRUMP: CHINA GOT BETTER DEAL IN PARIS CLIMATE AGREEMENT
TRUMP: CHINA GOT BETI'ER DEAL IN PARIS CLIMATE AGREEMENT. . Gov Jerry Brown (D) California CA GOV COMPARES TRUMP SUPPORTERS TO CAVE DWELLERS TRUMP: CHINA GOT BETTER DEAL IN PARIS CLIMATE AGREEMENT
TRUMP: CHINA GOT BETI'ER DEAL IN PARIS CLIMATE AGREEMENT. . Gov Jerry Brown (D) California CA GOV COMPARES TRUMP SUPPORTERS TO CAVE DWELLERS TRUMP: CHINA GOT BETTER DEAL IN PARIS CLIMATE AGREEMENT
trump: ch

In [57]:
txt

["trump: china got beti'er deal in paris climate agreement. . gov jerry brown (d) california ca gov compares trump supporters to cave dwellers trump: china got better deal in paris climate agreement",
 'macron speech at un. french president defends paris climate accord',
 "toys 'r' us files for bankruptcy in u.s.. toys 'r' us asia unaffected by proceedings syrian conflict. russia says syrian army fired on by us-backed rebels macron speech at un. french president defends paris climate...",
 'iraq - kurd referendum. iraqi-kurdish leader says vote could be delayed syria conflict. russia claims troops attacked by us backed rebels macron speech at un. french president defends paris climate accord',
 'macron speech at un. french president defends paris climate accord',
 'syria conflict. russia claims troops attacked by us backed rebels iraq - kurd referendum. iraqi-kurdish leader says vote could be delayed macron speech at un. french president defends paris climate...',
 'macron discusses is